# Intermediate PyTorch: Building architectures

## Every neural net requires the `init` method and the `forward` method. 
`init` : Define the kinds of layers from PyTorch you want to utilize
`forward` : Define the input feature matrix interaction with the layers

In [10]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms


### Building a model class

In [11]:

class CustomNet(nn.Module): # nn.Module is the base class for all neural network modules
    ## Copilot recommended
    def __init__(self):
        super(CustomNet, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 3)
        self.fc3 = nn.Linear(3, 1)

    def forward(self, x): # Defines the computation performed at every call , every neuron
        # Forward pass
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

    def predict(self, x):
        # Predicting the output
        pred = self.forward(x)
        return pred

    def get_weights(self):
        # Get the weights
        return self.fc1.weight, self.fc2.weight

    def get_bias(self):
        # Get the bias
        return self.fc1.bias, self.fc2.bias

### Preventing Exploding Gradients and Vanishing Gradients 
 - Use Batch Normalization : During training, we normalize the values of each layer neuron
 - Use elu instead of relu to prevent non-zero gradients for 0 valued outputs

In [12]:
# Sample Batch norm
class CustomNet(nn.Module): # nn.Module is the base class for all neural network modules
    ## Copilot recommended
    def __init__(self):
        super(CustomNet, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.bn1 = nn.BatchNorm1d(5)
        self.fc2 = nn.Linear(5, 3)
        self.bn2 = nn.BatchNorm1d(3)
        self.fc3 = nn.Linear(3, 1)

    def forward(self, x): # Defines the computation performed at every call , every neuron
        x = F.elu(self.fc1(x))
        x = self.bn1(x)
        x = F.elu(self.fc2(x))
        x = self.bn2(x)
        x = F.sigmoid(self.fc3(x))
        return x
    
    
    

### Convolution Neural Networks for images.
Why not linear layers ? Considering a single image as rowPixel x colPixel. 256 * 256 = ..some big number. Passed through this layer can cause large computation time for gradient updates and processing power as well.

Convolution makes use of a filter which is moved over the original image to extract it's feature map. Padding of 0 on the border of image is done to maintain spatial dimension and avoid information loss.

Usual process :

Image --> Convolution --> Activation --> MaxPooling( to reduce dimension ) --> Convolution --> Activation --> MaxPooling( to reduce dimension ) --> Fully Connected Linear layer

- `Convolution` : Changes number of output channels
- `MaxPool`     : Reduces the size of each channel

In [13]:
class Convolution_nn(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.feature_map = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1) ,# 3 input channels, 32 output channels, 3x3 kernel
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),)
        self.classifier = nn.Linear(64*16*16, num_classes) # 64*8*8 is the number of features extracted by the convolutional layers

    def forward(self, x):  
        # Pass input interactions
        x = self.feature_map(x)
        x = self.classifier(x)
        return x

### Data Augmentation: Transforms you can use on a datasets


In [14]:
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(35),
    transforms.RandomAutocontrast(),
    transforms.ToTensor(),
    transforms.Resize((64,64)),
])

### Metrics in torchmetrics


In [15]:
from torchmetrics import Precision, Recall

ModuleNotFoundError: No module named 'torchmetrics'